# LLM Workshop Notebook - OpenAI Version

**Author:** Aron Brand

**Copyright (c) 2024**

This notebook is an integral part of Aron Brand's LLM Workshop, designed to explore and utilize the capabilities of large language models (LLMs).

**License:**

This program is free software: you are encouraged to redistribute and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation. This software is provided under either version 3 of the License, or (at your discretion) any later version.

The program is distributed in the hope that it will be useful and informative. However, it comes with no warranty; not even the implied warranty of merchantability or fitness for a particular purpose. For more details, please refer to the GNU General Public License.

For a copy of the GNU General Public License, please visit [https://www.gnu.org/licenses/](https://www.gnu.org/licenses/).

# Install dependencies

In [1]:
%pip install openai langchain langchain-openai unstructured chromadb langchainhub --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 5.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.83
    Uninstalling langsmith-0.0.83:
      Successfully uninstalled langsmith-0.0.83
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.16
    Uninstalling langchain-core-0.1.16:
      Successfully uninstalle

## Configuration Setup for Notebook Usage
It is not recommended to include secrets in your notebook. Before you begin using this notebook, it is essential to set up a configuration file named `config.ini`. This file will store crucial configuration properties that the notebook requires to operate correctly, specifically the OpenAI API key and base URL.

### Creating the `config.ini` File

1. Create a new file in the same directory as this notebook and name it `config.ini`.

2. Open the file in a text editor.

3. Add the following structure to the file:

   ```ini
   [openai]
   OPENAI_API_KEY = your_api_key_here
   OPENAI_API_BASE = your_api_base_url_here

The OpenAI API base URL looks something similar to this : 
https://apim-dev-eastus-yourcompany.azure-api.net/Example-Project/openai


In [2]:
import configparser

# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the configuration file
config.read('config.ini')

# Access the values
OPENAI_API_KEY = config['openai']['OPENAI_API_KEY']
#OPENAI_API_BASE = config['openai']['OPENAI_API_BASE']

# Initialize ChatOpenAI

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo",
    temperature=0.2
)

# Your first LangChain "Hello World"

In [6]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [HumanMessage(content="Tell me a joke")]
response = llm.invoke(messages)

response.pretty_print() 

================================== Ai Message ==================================

Why did the scarecrow win an award? Because he was outstanding in his field!


# Add a System Message

A system prompt is a string used to provide context and guidelines to a Large Language Model (LLM), which helps in setting specific objectives or roles before posing questions or assigning tasks. Key components of system prompts include:

- **Task Directives:** Detailed instructions for the specific task at hand.
- **Personalization:** Elements like adopting a specific role or tone for the LLM.
- **Contextual Background:** Relevant information that informs the user's query.
- **Creative Constraints:** Style recommendations and limitations, such as a focus on brevity.
- **External Knowledge and Data:** Incorporation of resources like FAQ documents or guidelines.
- **Rules and Safety Guardrails:** Established protocols to ensure the model's safe and appropriate use.
- **Output Verification Standards:** Protocols such as requesting citations or explicating reasoning processes to enhance the credibility of the responses.


In [7]:
messages = [ 
  SystemMessage(content="Say the opposite of what the user says in a very impolite way"), 
  HumanMessage(content="Langchain is an awesome piece of software.") 
]

response = llm.invoke(messages)

response.pretty_print() 

================================== Ai Message ==================================

Langchain is a terrible piece of software.


# Continue a Chat

In [8]:
messages = [ 
	SystemMessage(content="Speak like a pirate"), 	
	HumanMessage(content="I''m Captain Aron. What''s your name?"), 
	AIMessage(content="Ahoy, Captain! Me name be ChatGPT, the savviest AI on the seven digital seas! How can I assist ye on this fine day?"),
	HumanMessage(content="Who named you that?") 
]

response = llm.invoke(messages)

response.pretty_print() 

================================== Ai Message ==================================

Arrr, me name be bestowed upon me by the scallywags who created me, matey. They thought it be a fitting moniker for a helpful AI like meself. Now, what be yer next command, Captain Aron?


# Extract data from text

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Role: Numeric fact extractor. Return data with no additional commentary. Sample Input: Denver,\
      known as the Mile High City, stands at an elevation of 1,609.6 meters above sea level. Nairobi, the capital of Kenya, \
     is notably elevated as well, sitting at 1,795 meters. Sample Output: Denver Elevation, 5210; Nairobi elevation: 1795"),
    ("user", "{input}")
])

chain = prompt | llm

response = chain.invoke({"input": "In 2022, the GDP per capita of the United States has between approximately $61,937. \
Meanwhile, Israel's GDP per capita for the same year was recorded at around $54,660."})

response.pretty_print() 

================================== Ai Message ==================================

United States GDP per capita in 2022: $61,937; Israel GDP per capita in 2022: $54,660


# Batch Processing

In [10]:
response = chain.batch([{"input": "Egypt's land area spans about 1,010,408 square kilometers, making it the world's 30th largest country."},
{"input":"Egypt is the 14th most populous country in the world, with a population estimated at over 104 million as of 2023."}])

response

[AIMessage(content='Egypt land area: 1,010,408 square kilometers.'),
 AIMessage(content='Egypt population: 104,000,000; Egypt global ranking by population: 14th')]

# LangChain Output Parsers

In [11]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = ChatPromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt | llm | output_parser

chain.invoke({"question": "When was OpenAI founded?"})


datetime.datetime(2015, 12, 11, 0, 0)

# Load and parse a web page with "Unstructured"

Unstructured supports loading of text files, powerpoints, html, pdfs, images, and more. 

In [12]:
from langchain_community.document_loaders import UnstructuredURLLoader

urls = ["https://www.ctera.com/all-products/"]

loader = UnstructuredURLLoader(urls=urls, headers={"User-Agent": "Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36"})
documents = loader.load()


# Split the web page to chunks

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = 1500,
    chunk_overlap  = 0
)

texts = text_splitter.split_documents(documents)

texts[:5]

[Document(page_content='Products overview\n\nPowering application infrastructure for the cloud\n\nDeliver and operate infrastructure for cloud applications rapidly, reliably and scalably with maximum efficiency and continuous optimization.\n\nRequest a demo\n\nGet the most out of your cloud investment\n\nOur solutions combine machine learning and analytics with decades of operations expertise to simplify, automate and optimize operations for your cloud infrastructure, freeing teams to focus on delivering impactful applications without being overburdened by operations and infrastructure.\n\nI think of Spot not just as a cost optimization platform, but as our container experts.\n\nSteve Evans,\n\nVP, Engineering Services\n\nSee case study\n\nHow our products can help\n\nEnsure availability & performance\n\nAvoid service disruption while utilizing a balanced blend of spot, reserved and on-demand instances without compromising availability and reliability. Spot delivers availability and pe

# Summarize using Map Reduce Chain

In [14]:
from langchain.chains.summarize import load_summarize_chain

map_prompt_template = """
                      Write a summary of this chunk of text that includes all the main points and any important details. 
                      Ignore code snippets or technical markup. Include all important names and terms mentioned.
                      {text}
                      """

map_prompt = ChatPromptTemplate.from_template(map_prompt_template)

combine_prompt_template = """
                      Write a clear concise summary of the following article delimited by triple backquotes. Use elegant prose that is not repetitive.
                      Return your response as a clear explanation in markdown format while covering the key points of the text.
                      Use a flowing clear article style.
                      ```{text}```
                      SHORT ARTICLE:
                      """

combine_prompt = ChatPromptTemplate.from_template(
    template=combine_prompt_template
)

chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
    verbose=True
)
result = chain.invoke(texts)

result



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
                      Write a summary of this chunk of text that includes all the main points and any important details. 
                      Ignore code snippets or technical markup. Include all important names and terms mentioned.
                      Products overview

Powering application infrastructure for the cloud

Deliver and operate infrastructure for cloud applications rapidly, reliably and scalably with maximum efficiency and continuous optimization.

Request a demo

Get the most out of your cloud investment

Our solutions combine machine learning and analytics with decades of operations expertise to simplify, automate and optimize operations for your cloud infrastructure, freeing teams to focus on delivering impactful applications without being overburdened by operations and infrastructure.

I think of Spot not just as a cost optimization platform, but as

{'input_documents': [Document(page_content='Products overview\n\nPowering application infrastructure for the cloud\n\nDeliver and operate infrastructure for cloud applications rapidly, reliably and scalably with maximum efficiency and continuous optimization.\n\nRequest a demo\n\nGet the most out of your cloud investment\n\nOur solutions combine machine learning and analytics with decades of operations expertise to simplify, automate and optimize operations for your cloud infrastructure, freeing teams to focus on delivering impactful applications without being overburdened by operations and infrastructure.\n\nI think of Spot not just as a cost optimization platform, but as our container experts.\n\nSteve Evans,\n\nVP, Engineering Services\n\nSee case study\n\nHow our products can help\n\nEnsure availability & performance\n\nAvoid service disruption while utilizing a balanced blend of spot, reserved and on-demand instances without compromising availability and reliability. Spot delivers

In [15]:
from IPython.display import Markdown

display(Markdown(result['output_text']))


The article provides an overview of products designed to power application infrastructure for the cloud, aiming to deliver and operate cloud applications rapidly, reliably, and scalably with maximum efficiency and continuous optimization. These products combine machine learning and analytics with operations expertise to simplify, automate, and optimize operations for cloud infrastructure, allowing teams to focus on delivering impactful applications without being burdened by operations and infrastructure. Steve Evans, VP of Engineering Services, praises the platform as container experts. The products ensure availability and performance by utilizing a balanced blend of spot, reserved, and on-demand instances, continuously optimizing cloud resources to reduce overprovisioning and wasted resources, ultimately aiming to reduce cloud infrastructure costs. Spot offers insights, guidance, and automation across all cloud infrastructures to optimize costs, utilizing machine learning and analytics to predict interruptions and auto-replace instances. By using Spot, users can save up to 90% on cloud costs without changing their applications. Various tools and solutions for optimizing cloud costs, infrastructure automation, and observability are discussed, including Eco, CloudCheckr, Elastigroup, Ocean, Ocean CD, Spot Security, and Cloud Insights. The article also mentions Instaclustr's open-source application services and encourages readers to try Spot by NetApp's cloud cost optimization tools for comprehensive visibility and maximum cost-efficiency.

# Summarize with Refine Chain

In [16]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = ChatPromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final detailed summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary clearly"
    "If the context isn't useful, return the original summary. Use markdown format."
)
refine_prompt = ChatPromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)

result = chain.invoke(texts)

result

{'input_documents': [Document(page_content='Products overview\n\nPowering application infrastructure for the cloud\n\nDeliver and operate infrastructure for cloud applications rapidly, reliably and scalably with maximum efficiency and continuous optimization.\n\nRequest a demo\n\nGet the most out of your cloud investment\n\nOur solutions combine machine learning and analytics with decades of operations expertise to simplify, automate and optimize operations for your cloud infrastructure, freeing teams to focus on delivering impactful applications without being overburdened by operations and infrastructure.\n\nI think of Spot not just as a cost optimization platform, but as our container experts.\n\nSteve Evans,\n\nVP, Engineering Services\n\nSee case study\n\nHow our products can help\n\nEnsure availability & performance\n\nAvoid service disruption while utilizing a balanced blend of spot, reserved and on-demand instances without compromising availability and reliability. Spot delivers

In [17]:
display(Markdown(result['output_text']))

Spot by NetApp offers solutions to power cloud applications infrastructure efficiently and effectively. Their products combine machine learning and analytics to simplify operations and optimize cloud infrastructure. By utilizing a balanced blend of spot, reserved, and on-demand instances, Spot ensures availability and performance while reducing costs. Their technology continuously optimizes cloud resources to avoid overprovisioning and wasted resources. Spot also provides insights, guidance, and automation to help users save up to 90% on cloud infrastructure costs without changing or rearchitecting their applications. Their sophisticated machine learning and analytics predict interruptions and auto-replace instances to maintain availability and performance. Additionally, Spot offers commitment optimization with Eco and cloud cost management with CloudCheckr. They provide infrastructure automation and optimization solutions such as Elastigroup for virtual machines, Ocean for containers and Kubernetes, Ocean CD for containerized application delivery, and Spot Security for security analysis. Cloud Insights enables infrastructure observability through observability, analytics, and insights to monitor, troubleshoot, optimize, and secure resources at scale. Spot also offers open-source application services through the Instaclustr portfolio, including Apache Cassandra, Apache Kafka, PostgreSQL, Redis, OpenSearch, and more. Customers can try Spot by NetApp's cloud cost optimization tools for comprehensive visibility and maximum cost-efficiency by requesting a trial.

# Vectorize text to embeddings

In [19]:
from langchain_openai import OpenAIEmbeddings 

embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
#    openai_api_base=OPENAI_API_BASE,
    model="text-embedding-3-small"
)


In [20]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embedding=embeddings)

# Simple RAG Q&A pipeline

In [21]:
from langchain.chains import RetrievalQA
from langchain_core.vectorstores import VectorStoreRetriever

retriever = VectorStoreRetriever(vectorstore=db)

prompt_template = """Human: You are a very honest, persuasive salesperson. Use the following pieces of context 
to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, 
don't try to make up an answer.
{context}
Question: {question}
Assistant:"""
PROMPT = ChatPromptTemplate.from_template(template=prompt_template)

qa = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=retriever,
return_source_documents=True,
chain_type_kwargs={"prompt": PROMPT}
)


In [22]:
from IPython.display import Markdown, display

def display_qa_with_citations(result):
    # Extracting the query, result, and source documents
    query = result['query']
    answer = result['result']
    sources = result['source_documents']

    # Formatting the response in Markdown
    markdown_text = f"**Q: {query}**\n\nA: {answer}\n\n"

    # Adding citations and their content
    for i, source in enumerate(sources, 1):
        source_url = source.metadata['source']
        markdown_text += f"**Citation {i}:** [Source]({source_url})\n\n"

    # Displaying the formatted text
    display(Markdown(markdown_text))


In [23]:
result = qa.invoke({"query": "How can you reduce my cloud costs?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "What automations do you offer?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "How many cars do you sell per year?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)


Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


**Q: How can you reduce my cloud costs?**

A: You can reduce your cloud costs by using Spot by NetApp's cloud cost optimization tools, which provide insights, guidance, and automation to help you save up to 90% without changing or rearchitecting your applications. Spot ensures availability and performance, continuously optimizes cloud resources, and helps you understand and optimize your cloud spend across all your cloud infrastructures.

**Citation 1:** [Source](https://spot.io/products/)

**Citation 2:** [Source](https://spot.io/products/)

**Citation 3:** [Source](https://spot.io/products/)



Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


**Q: What automations do you offer?**

A: We offer a variety of automation solutions, including Elastigroup for virtual machines, Ocean for containers and Kubernetes, Ocean CD for containerized application delivery, and Spot Security for security analysis and threat reduction.

**Citation 1:** [Source](https://spot.io/products/)

**Citation 2:** [Source](https://spot.io/products/)

**Citation 3:** [Source](https://spot.io/products/)



Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


**Q: How many cars do you sell per year?**

A: I don't have that information.

**Citation 1:** [Source](https://spot.io/products/)

**Citation 2:** [Source](https://spot.io/products/)

**Citation 3:** [Source](https://spot.io/products/)



# ADVANCED: Function Calling

Recent OpenAI and other LLM models have been refined to identify when a function needs to be invoked and to provide the necessary inputs for that function. Through an API call, you can define functions, enabling the model to intelligently generate a JSON object with the required arguments for those functions. The primary aim of the Function APIs is to ensure more consistent and reliable results compared to standard chat API

In [27]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.utils.function_calling import convert_to_openai_function

@tool
def multiply(a: float, b: float) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = [multiply, get_word_length]

model_with_tools = llm.bind(functions=[convert_to_openai_function(t) for t in tools])

message = model_with_tools.invoke([HumanMessage(content="How much is 23.1 times ten")])

message

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"a":23.1,"b":10}', 'name': 'multiply'}})

In [25]:
model_with_tools.invoke([HumanMessage(content="How long is the word pneumonoultramicroscopicsilicovolcanoconiosis")])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"word":"pneumonoultramicroscopicsilicovolcanoconiosis"}', 'name': 'get_word_length'}})

# ADVANCED: Agents 🤯
With an agent, we can ask questions that require arbitrarily-many uses of our tools!

In [26]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "Take the length of the longest word in English and multiply it by 192.7, then square the result"
    }
)





> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'pneumonoultramicroscopicsilicovolcanoconiosis'}`


45
Invoking: `multiply` with `{'a': 45, 'b': 192.7}`


8671.5
Invoking: `multiply` with `{'a': 8671.5, 'b': 8671.5}`


75194912.25The length of the longest word in English is 45. 

Multiplying it by 192.7 gives 8671.5. 

Squaring this result gives 75,194,912.25.

> Finished chain.


{'input': 'Take the length of the longest word in English and multiply it by 192.7, then square the result',
 'output': 'The length of the longest word in English is 45. \n\nMultiplying it by 192.7 gives 8671.5. \n\nSquaring this result gives 75,194,912.25.'}